In [1]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


In [2]:
pip install pandas_ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=ae240b0883c7b66c39d8b20ab5a557d663ed696a509573d1cff9818e350848dd
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install pyngrok

In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
from alpha_vantage.fundamentaldata import FundamentalData
from stocknews import StockNews
import pandas_ta as ta
from pandas.tseries.holiday import USFederalHolidayCalendar

# Function to set a background image
def set_background(background_image):
    st.markdown(
        f"""
        <style>
            .reportview-container {{
                background: url("{background_image}") no-repeat center center;
                background-size: cover;
            }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Set background image
set_background("https://images.unsplash.com/photo-1578861082890-f9211a299fb6")

# Add a famous quote
st.markdown(
    """
    <div style='text-align:center; padding: 20px;'>
        <h2 style='color: white;'>YOUR TRADING IS GOING TO PAY OFF SOONER THAN YOU THINK. BE PATIENT.</h2>
    </div>
    """,
    unsafe_allow_html=True
)

st.title('Stock Dashboard')

# List of stocks
stocks = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'FB', 'NFLX', 'TSLA', 'BA', 'IBM', 'NVDA',
          'INTC', 'CSCO', 'PYPL', 'ADBE', 'NFLX', 'ASML', 'CMCSA', 'PEP', 'AVGO', 'TXN']

# Function to analyze stocks and recommend the one with the least error
def recommend_stock(stocks):
    min_error = float('inf')
    recommended_stock = None
    for stock in stocks:
        try:
            data = yf.download(stock, period="1d")
            error = np.std(data['Close']) / np.mean(data['Close'])
            if error < min_error:
                min_error = error
                recommended_stock = stock
        except Exception as e:
            print(f"Error downloading data for {stock}: {e}")
    return recommended_stock

# Recommended stock to buy
recommended_stock = recommend_stock(stocks)
st.sidebar.write(f"Recommended Stock to Buy: {recommended_stock}")

# Input sidebar for ticker, start and end date
selected_stock = st.sidebar.selectbox('Select Stock', stocks, index=0)
start_date = st.sidebar.text_input('Start Date', '2001-01-01')
end_date = st.sidebar.text_input('End Date', '2024-03-29')

# Download data function call
data = yf.download(selected_stock, start=start_date, end=end_date)

# Plot the stock price line chart
fig = px.line(data, x=data.index, y=data['Adj Close'], title=selected_stock)
st.plotly_chart(fig)

# Display Price Movements
st.header('Price Movements')
data2 = data
data2['% Change'] = data['Adj Close'] / data['Adj Close'].shift(1) - 1
data2.dropna(inplace=True)
st.write(data2)

annual_return = data2['% Change'].mean() * 252 * 100
st.write('Annual Return is', annual_return, '%')
stdev = np.std(data2['% Change']) * np.sqrt(252)
st.write('Standard Deviation is ', stdev * 100, '%')

# Fundamental Data
st.header('Fundamental Data')
key = 'VCZRSZJCMBEC02D1'  # Add your Alpha Vantage API key here
fd = FundamentalData(key, output_format='pandas')
st.subheader('Balance Sheet')
balance_sheet = fd.get_balance_sheet_annual(selected_stock)[0]
bs = balance_sheet.T[2:]
bs.columns = list(balance_sheet.T.iloc[0])
st.write(bs)

st.subheader('Income Statement')
income_statement = fd.get_income_statement_annual(selected_stock)[0]
isl = income_statement.T[2:]
isl.columns = list(income_statement.T.iloc[0])
st.write(isl)

st.subheader('Cash Flow Statement')
cash_flow = fd.get_cash_flow_annual(selected_stock)[0]
cf = cash_flow.T[2:]
cf.columns = list(cash_flow.T.iloc[0])
st.write(cf)

# Top 10 News for Recommended Stock
st.header('Top 10 News for Recommended Stock')
sn = StockNews(recommended_stock)
df_news = sn.read_rss()
for i in range(10):
    st.subheader(f'News {i+1}')
    st.write(df_news['published'][i])
    st.write(df_news['title'][i])
    st.write(df_news['summary'][i])
    title_sentiment = df_news['sentiment_title'][i]
    st.write(f'Title Sentiment {title_sentiment}')
    news_sentiment = df_news['sentiment_summary'][i]
    st.write(f'News Sentiment {news_sentiment}')

# Technical Analysis Dashboard
st.header('Technical Analysis Dashboard')
indicators = ['MACD', 'RSI', 'Bollinger Bands', 'ATR', 'OBV', 'AD', 'CCI', 'Chaikin Money Flow', 'VWAP']
selected_indicator = st.selectbox('Select Technical Indicator', options=indicators)

if selected_indicator == 'MACD':
    indicator_data = ta.macd(data['Close'])
elif selected_indicator == 'RSI':
    indicator_data = ta.rsi(data['Close'])
elif selected_indicator == 'Bollinger Bands':
    indicator_data = ta.bbands(data['Close'])
elif selected_indicator == 'ATR':
    indicator_data = ta.atr(data['High'], data['Low'], data['Close'])
elif selected_indicator == 'OBV':
    indicator_data = ta.obv(data['Close'], data['Volume'])
elif selected_indicator == 'AD':
    indicator_data = ta.ad(data['High'], data['Low'], data['Close'], data['Volume'])
elif selected_indicator == 'CCI':
    indicator_data = ta.cci(data['High'], data['Low'], data['Close'])
elif selected_indicator == 'Chaikin Money Flow':
    indicator_data = ta.cmf(data['High'], data['Low'], data['Close'], data['Volume'])
elif selected_indicator == 'VWAP':
    indicator_data = ta.vwap(data['High'], data['Low'], data['Close'], data['Volume'])

st.line_chart(indicator_data)
st.write(indicator_data)

# Portfolio Summary
st.header('Portfolio Summary')
if st.checkbox("Show Portfolio Summary"):
    # Example data (replace with your actual portfolio data)
    portfolio_value = 1000000  # Total portfolio value
    cash = 200000  # Cash balance
    stocks_value = 800000  # Value of stocks
    total_return = 0.15  # Total return (15%)
    annual_return = 0.10  # Annualized return (10%)
    volatility = 0.20  # Portfolio volatility (20%)

    st.subheader("Portfolio Value")
    st.write(f"Total Portfolio Value: ${portfolio_value:,.2f}")
    st.write(f"Cash: ${cash:,.2f}")
    st.write(f"Stocks Value: ${stocks_value:,.2f}")

    st.subheader("Performance Metrics")
    st.write(f"Total Return: {total_return * 100:.2f}%")
    st.write(f"Annualized Return: {annual_return * 100:.2f}%")
    st.write(f"Portfolio Volatility: {volatility * 100:.2f}%")

# Watchlist
st.header('Watchlist')
if st.checkbox("Show Watchlist"):
    # Example data (replace with your actual watchlist data)
    watchlist = {
        'AAPL': {'price': 150.50, 'change': 1.25, 'percent_change': 0.84},
        'GOOG': {'price': 2800.75, 'change': -5.50, 'percent_change': -0.20},
        'MSFT': {'price': 250.20, 'change': 2.80, 'percent_change': 1.13},
        # Add more stocks as needed
    }

    st.subheader("Stocks in Watchlist")
    for stock, data in watchlist.items():
        st.write(f"{stock}:")
        st.write(f"Price: ${data['price']:.2f}")
        st.write(f"Change: ${data['change']:.2f}")
        st.write(f"Percent Change: {data['percent_change']:.2f}%")

# Market Indices
st.header('Market Indices')
if st.checkbox("Show Market Indices"):
    # Example data (replace with your actual market indices data)
    sp500_value = 4000  # S&P 500 index value
    dow_value = 33000  # Dow Jones Industrial Average index value
    nasdaq_value = 13000  # NASDAQ index value

    st.subheader("Market Indices")
    st.write(f"S&P 500 Index: {sp500_value}")
    st.write(f"Dow Jones Industrial Average: {dow_value}")
    st.write(f"NASDAQ Composite: {nasdaq_value}")

# Individual Stock Performance
st.header('Individual Stock Performance')
if st.checkbox("Show Individual Stock Performance"):
    stocks = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'FB', 'NFLX', 'TSLA', 'BA', 'IBM', 'NVDA',
              'INTC', 'CSCO', 'PYPL', 'ADBE', 'NFLX', 'ASML', 'CMCSA', 'PEP', 'AVGO', 'TXN']

    for symbol in stocks:
        # Replace this with your actual stock data retrieval and calculations
        # For now, we'll use some example data
        price = np.random.uniform(100, 500)  # Example price
        change = np.random.uniform(-5, 5)  # Example change
        change_percentage = (change / price) * 100

        st.subheader(symbol)
        st.write(f"Price: ${price:.2f}")
        st.write(f"Change: ${change:.2f}")
        st.write(f"Change %: {change_percentage:.2f}%")

# Charts and Graphs
st.header('Charts and Graphs')
if st.checkbox("Show Charts and Graphs"):
    # Example data (replace with your actual data)

    # Generate sample data
    df = pd.DataFrame({
        'Date': pd.date_range(start='2022-01-01', end='2022-12-31'),
        'Stock_A': np.random.normal(loc=100, scale=10, size=365),
        'Stock_B': np.random.normal(loc=80, scale=8, size=365)
    })

    # Line chart
    st.subheader("Stock Prices Over Time")
    st.line_chart(df.set_index('Date'))

    # Bar chart
    st.subheader("Stock Performance Comparison")
    st.bar_chart(df[['Stock_A', 'Stock_B']].mean())

    # Scatter plot
    st.subheader("Correlation Between Stock A and Stock B")
    st.write("This scatter plot shows the correlation between Stock A and Stock B.")
    st.scatter_chart(df, x='Stock_A', y='Stock_B')

# Financial Statements
st.header('Financial Statements')
if st.checkbox("Show Financial Statements"):
    # Example data (replace with your actual financial statements data)
    # Assume you have fetched balance sheet, income statement, and cash flow statement data
    balance_sheet_data = {
        'Assets': {'Cash': 100000, 'Accounts Receivable': 200000, 'Inventory': 150000},
        'Liabilities': {'Accounts Payable': 75000, 'Loans Payable': 50000, 'Accrued Expenses': 25000},
        'Equity': {'Common Stock': 100000, 'Retained Earnings': 150000}
    }

    income_statement_data = {
        'Revenue': 500000,
        'Expenses': {'Cost of Goods Sold': 250000, 'Operating Expenses': 100000, 'Interest Expense': 25000},
        'Net Income': 125000
    }

    cash_flow_statement_data = {
        'Operating Activities': {'Net Income': 125000, 'Depreciation': 50000, 'Changes in Working Capital': 25000},
        'Investing Activities': {'Purchase of Equipment': -75000, 'Sale of Investments': 50000},
        'Financing Activities': {'Issuance of Common Stock': 100000, 'Repayment of Loans': -50000},
        'Net Change in Cash': 175000
    }

    # Display Balance Sheet
    st.subheader('Balance Sheet')
    for category, items in balance_sheet_data.items():
        st.write(category)
        for item, value in items.items():
            st.write(f"- {item}: ${value:,.2f}")

    # Display Income Statement
    st.subheader('Income Statement')
    for item, value in income_statement_data.items():
        if isinstance(value, dict):
            st.write(item)
            for sub_item, sub_value in value.items():
                st.write(f"- {sub_item}: ${sub_value:,.2f}")
        else:
            st.write(f"{item}: ${value:,.2f}")

    # Display Cash Flow Statement
    st.subheader('Cash Flow Statement')
    for category, items in cash_flow_statement_data.items():
        if isinstance(items, dict):
            st.write(category)
            for sub_item, sub_value in items.items():
                st.write(f"- {sub_item}: ${sub_value:,.2f}")
        else:
            st.write(f"{category}: ${items:,.2f}")



# Market Calendar
st.header('Market Calendar')
if st.checkbox("Show Market Calendar"):
    # Define date range
    start_date = pd.Timestamp.today()
    end_date = start_date + pd.Timedelta(days=30)  # Display events for the next 30 days

    # Generate US Federal Holiday Calendar
    calendar = USFederalHolidayCalendar()
    holidays = calendar.holidays(start=start_date, end=end_date)

    # Display holidays
    st.subheader("Upcoming Holidays")
    if len(holidays) > 0:
        for holiday in holidays:
            st.write(f"- {holiday.strftime('%Y-%m-%d')} : {holiday}")
    else:
        st.write("No upcoming holidays in the next 30 days.")


Writing app.py


In [5]:
pip install pandas_market_calendars


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.7/191.7 kB 3.3 MB/s eta 0:00:00


In [6]:
!pip install alpha_vantage


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 24.6 MB/s eta 0:00:00


In [9]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 10.2 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.1-cp310-cp310-linux_x86_64.whl size=726071 sha256=947fbeb747a892e76fe80433923b4db0fc7400dc5182a7d8d0375c8ba6155118
  Stored in directory: /root/.cache/pip/wheels/d7/35/5c/1374782be033462df5f40174d8d879519d64ed8c25a1977554
Successfully built peewee


In [7]:
!pip install stocknews

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stocknews: filename=stocknews-0.9.11-py3-none-any.whl size=5396 sha256=299d5f7a0dad048c578c5efe867722fe62ee03631e2a2041c4ca988c75c4f99c
  Stored in directory: /root/.cache/pip/wheels/9e/59/42/c5ab3171c3509ff8bc6aafbc089fe826eacf531ef7f7803ff6
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=bc52dac2bb6c9a1b756509d71bb3934c1dea44772356c02b2ec0d162c0cd4c0b
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built stocknews sgmllib3k


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.238.1.90:8501

npx: installed 22 in 2.876s
your url is: https://major-parts-take.loca.lt
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['FB']: Exception('%ticker%: No data found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*******

In [ ]:
!pip install alpha_vantage


In [ ]:
!pip install stocknews

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stocknews: filename=stocknews-0.9.11-py3-none-any.whl size=5396 sha256=35aee0f9df40fd4de5fc43100345331fc7a577cba48dca3e925ed25bc5ca18a6
  Stored in directory: /root/.cache/pip/wheels/9e/59/42/c5ab3171c3509ff8bc6aafbc089fe826eacf531ef7f7803ff6
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=d5d61dd33ed33560f2eec047ac96ad6a2cc5331506d03a66dd4937cd089d351b
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built stocknews sgmllib3k
